In [1]:
import vtk
import pandas as pd
import numpy as np

df = pd.read_csv("../data/box_list.csv")

#Points,Cellsを作成
points = vtk.vtkPoints()
cells = vtk.vtkCellArray()

#scale_factor(csvがm表記の場合mmに変換する.)
scale_factor = 1000

for i, row in df.iterrows():
    x, y, z = row['x'], row['y'], row['z']
    lx, ly, lz = row['lx'], row['ly'], row['lz']

    #ボックスの頂点をnumpyで計算
    vertices = np.array([
        [x - lx/2, y - ly/2, z - lz/2],
        [x + lx/2, y - ly/2, z - lz/2],
        [x + lx/2, y + ly/2, z - lz/2],
        [x - lx/2, y + ly/2, z - lz/2],
        [x - lx/2, y - ly/2, z + lz/2],
        [x + lx/2, y - ly/2, z + lz/2],
        [x + lx/2, y + ly/2, z + lz/2],
        [x - lx/2, y + ly/2, z + lz/2],
    ])* scale_factor

    point_ids = []
    
    #ポイントの挿入
    for v in vertices:
        point_id = points.InsertNextPoint(v.tolist())
        point_ids.append(point_id)

    #六面体セルを作成
    cell = vtk.vtkHexahedron()  #vtkVoxelからvtkHexahedronに変更
    for j, pid in enumerate(point_ids):
        cell.GetPointIds().SetId(j, pid)
    cells.InsertNextCell(cell)

grid = vtk.vtkUnstructuredGrid()
grid.SetPoints(points)
grid.SetCells(vtk.VTK_HEXAHEDRON, cells)  #VTK_HEXAHEDRONを指定

writer = vtk.vtkUnstructuredGridWriter()
writer.SetFileName("../data/output.vtk")
writer.SetInputData(grid)
writer.Write()

1